# Longitudinal Analysis of MRI Scans of Multiple Sclerosis Patients

The purpose of this notebook is to document the process to train BIANCA and LOCATE, and to have a script that allows re-doing the process automatically without having to manually sort, rename or adjust any files.

##### This Jupyter Notebook is accompanied with a Graphical User Interface that simply allows the utilization of the models trained in this notebook in the form of an interactive application. Please feel free to use that!



# Training of BIANCA

Before proceeding to the code part, it is essential to understand the steps that will be required for training BIANCA, and then LOCATE. We will be starting off with very little manual data renaming, which will be used to train BIANCA, and then all of it will be used for LOCATE

Following the training practical [https://open.win.ox.ac.uk/pages/fslcourse/practicals/seg_struc/index.html#bianca], we will be needing the following files for each subject, regardless of how many subjects there are:
- Flair Brain Image
- Flair Brian Lesion Mask 

The terms 'Optional' are used so that, in the case of good quality Brain Images, we can use the additional files as parameters to BIANCA to support the Model in its output.

## Pre-requisities:

To start off, we will be needing the **Flair Brain Image**, **Flair Brain Lesion Mask**, and the **T1 Brain Image** (Optional) for each subject. 
Once we have those, we can proceed and extract the remaining by registration and resampling. 

Another condition to ensure the proper functioning of the code blocks in this Jupyter Notebook is to have a folder that contains the names of the **Flair Brain Image**, **Flair Brain Lesion Mask**, and the **T1 Brain Image** with the following convention and structure:

**ms_0xx_tpy_z.nii.gz**
(x = subject number, y = timepoint, z = file name)
- As an example:
    - ms_002_tp1_flair_brain.nii.gz 
        - (having 'flair_brain' in the name is necessary)
    - ms_002_tp1_lesion_mask.nii.gz 
        - (having 'mask' in the name is necessary)
    - ms_002_tp1_T1_brain.nii.gz    
        - (having 'T1' in the name is necessary)

### An example of a folder I created for the test run is as follows:


<img src="notebook_images/file_structure.png" alt="Drawing" style="width: 800px;"/>


## First Step: Resampling all Images

One thing to note is that BIANCA requires all the **Flair Brain Image(s)** to be in the same dimensions, therefore we will figure out the most common/ occuring dimensions and will then resample all other images to be of that very dimension.

### Getting the dimensions for all the flair brain Images

#### First, we get the folder path where all the images are stored

In [4]:
def get_directory(type):
    from tkinter import Tk, filedialog

    root = Tk() # pointing root to Tk() to use it as Tk() in program.
    root.withdraw() # Hides small tkinter window.
    root.attributes('-topmost', True) # Opened windows will be active. above all windows despite of selection.
    
    if(type == "folder"):
        open_file = filedialog.askdirectory() # Returns opened path as str
    
    elif(type=="file"):
        open_file = filedialog.askopenfilename() # Returns opened path as str
    
    else:
        print("Invalid File Type")
        return 0

    return(open_file) 

In [2]:
open_file = get_directory("folder")
print(open_file)

/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data


#### Ensuring the correct folder is selected by printing out the names of all the files within the folder

In [3]:
import os
files = os.listdir(open_file) # returns list
print(files)

['ms_010_tp1_flair_brain.nii.gz', 'ms_005_tp1_t1_to_flair.nii.gz', 'ms_008_tp1_lesion_mask.nii.gz', 'ms_002_tp1_lesion_mask.nii.gz', 'ms_008_tp1_t1_to_flair.nii.gz', 'ms_002_tp1_t1_to_flair.nii.gz', 'ms_001_tp1_t1_to_flair.nii.gz', '.DS_Store', 'ms_006_tp1_lesion_mask.nii.gz', 'ms_006_tp1_t1_to_flair.nii.gz', 'ms_001_tp1_lesion_mask.nii.gz', 'ms_007_tp1_t1_to_flair.nii.gz', 'ms_007_tp1_lesion_mask.nii.gz', 'ms_004_tp1_lesion_mask.nii.gz', 'ms_003_tp1_t1_to_flair.nii.gz', 'ms_009_tp1_t1_to_flair.nii.gz', 'ms_003_tp1_lesion_mask.nii.gz', 'ms_009_tp1_lesion_mask.nii.gz', 'ms_004_tp1_t1_to_flair.nii.gz', 'Training-2023-04-02', 'ms_003_tp1_flair_brain.nii.gz', 'ms_009_tp1_flair_brain.nii.gz', 'ms_004_tp1_flair_brain.nii.gz', 'ms_005_tp1_lesion_mask.nii.gz.nii.gz', 'ms_007_tp1_flair_brain.nii.gz', 'ms_001_tp1_flair_brain.nii.gz', 'ms_006_tp1_flair_brain.nii.gz', 'Training-2023-02-19', 'ms_010_tp1_t1_to_flair.nii.gz', 'ms_005_tp1_flair_brain.nii.gz', 'ms_008_tp1_flair_brain.nii.gz', 'ms_002_t

#### Second, we create 3 different lists, and then store the files for the Flair Brain, Lesion Mask and T1 Brain images respectively

In [4]:
flairs = []
masks = []
t1s = []

for file in files:
#     print(file)
    if 'flair_brain' in file:
        flairs.append(file)
    elif 'mask' in file:
        masks.append(file)
    elif 'T1' in file:
        t1s.append(file)

In [5]:
print("All the Flair Brain Images are:")
print(flairs)
print()
print("All the Lesion Mask Images are:")
print(masks)
print()
print("All the T1 Brain Images are:")
print(t1s)

All the Flair Brain Images are:
['ms_010_tp1_flair_brain.nii.gz', 'ms_003_tp1_flair_brain.nii.gz', 'ms_009_tp1_flair_brain.nii.gz', 'ms_004_tp1_flair_brain.nii.gz', 'ms_007_tp1_flair_brain.nii.gz', 'ms_001_tp1_flair_brain.nii.gz', 'ms_006_tp1_flair_brain.nii.gz', 'ms_005_tp1_flair_brain.nii.gz', 'ms_008_tp1_flair_brain.nii.gz', 'ms_002_tp1_flair_brain.nii.gz']

All the Lesion Mask Images are:
['ms_008_tp1_lesion_mask.nii.gz', 'ms_002_tp1_lesion_mask.nii.gz', 'ms_006_tp1_lesion_mask.nii.gz', 'ms_001_tp1_lesion_mask.nii.gz', 'ms_007_tp1_lesion_mask.nii.gz', 'ms_004_tp1_lesion_mask.nii.gz', 'ms_003_tp1_lesion_mask.nii.gz', 'ms_009_tp1_lesion_mask.nii.gz', 'ms_005_tp1_lesion_mask.nii.gz.nii.gz']

All the T1 Brain Images are:
[]


### Checking the most frequently occuring dimension

In [23]:
def run_command(terminal_command):
    p = os.popen(terminal_command)
    if p:
        return p.read()
    else:
        return "Command Invalid"

In [32]:
dimensions = []
directory_1 = open_file.replace(' ', '\ ')

def check_dimensions(file_name, printvals):
        output = run_command(f"cd {directory_1} && fslinfo {file_name}")
        output = output.replace("\t", "\n")
        file_info = output.split("\n")
        while '' in file_info:
            file_info.remove('')
        #print(file_info)
        dim1 = file_info[3]
        dim2 = file_info[5]
        dim3 = file_info[7]
        if printvals== True:
            print("Dimensions of: ", file_name)
            print(dim1,dim2,dim3)
        return(dim1,dim2,dim3)

for i in range(len(flairs)):
    dimensions.append(check_dimensions(flairs[i], True))

Dimensions of:  ms_010_tp1_flair_brain.nii.gz
256 256 35
Dimensions of:  ms_003_tp1_flair_brain.nii.gz
256 256 35
Dimensions of:  ms_009_tp1_flair_brain.nii.gz
256 256 35
Dimensions of:  ms_004_tp1_flair_brain.nii.gz
256 256 35
Dimensions of:  ms_007_tp1_flair_brain.nii.gz
256 256 35
Dimensions of:  ms_001_tp1_flair_brain.nii.gz
256 256 35
Dimensions of:  ms_006_tp1_flair_brain.nii.gz
256 256 35
Dimensions of:  ms_005_tp1_flair_brain.nii.gz
256 256 35
Dimensions of:  ms_008_tp1_flair_brain.nii.gz
256 256 35
Dimensions of:  ms_002_tp1_flair_brain.nii.gz
256 256 35


In [7]:
def most_frequent(List):
    return max(set(List), key = List.count)
 
most_freq = most_frequent(dimensions)
print("The Most Frequently Occuring Dimension is: ")
print(most_freq[0],"x", most_freq[1],"x", most_freq[2])

The Most Frequently Occuring Dimension is: 
256 x 256 x 35


### Picking an Image with the most frequently occuring dimension for reference purposes

#### Index for the most frequently occuring dimension

In [33]:
ind = dimensions.index(most_freq)

#### Getting the corresponding Value from the *flairs* list for the corresponding image

In [34]:
ref_img = flairs[ind]
print(ref_img)

ms_010_tp1_flair_brain.nii.gz


### Create a subdirectory 'Training-{DATE}' to store all the resampled files, which shall be used for training purposes

#### Function to create a Directory

In [4]:
def create_directory(folder_path, directory_name):
    path = os.path.join(folder_path, directory_name)
    try:
        os.mkdir(path)
    except OSError as error:
        print(error)

In [36]:
from datetime import date
today = date.today()
directory_name = 'Training-' + str(today)

In [37]:
create_directory(open_file, directory_name)

### Within the subdirectory, we create multiple subdirectories for each subject

In [38]:
for file in flairs:
    print("_".join(file.split('_')[0:3]))
    folder_name = "_".join(file.split('_')[0:3])
    
    create_directory(open_file + '/' + directory_name, folder_name)

ms_010_tp1
ms_003_tp1
ms_009_tp1
ms_004_tp1
ms_007_tp1
ms_001_tp1
ms_006_tp1
ms_005_tp1
ms_008_tp1
ms_002_tp1


### Now, we should essentially have a directory structure like the following:

<img src="notebook_images/new_folders.png" alt="Drawing" style="width: 700px;"/>


### Using the reference Image, we iterate through all the flair brain images with different dimensions and resample them to be in the same dimensions, and then store them into their respective subdirectory

#### We do the same for the corresponding Lesion Masks to those Flair Brain Images since they should be resampled into the same dimensions as their corresponding flairs

#### However, out of all the subjects, one of the Subjects will be solely for testing purposes and will not have an existing flair lesion mask. Therefore, since it can be any, we will ask the user for the subject number and timepoint for the test subject

In [39]:
training_folder = input("Please enter the Test Subject as ms_0xx_tpx (example: ms_003_tp1) ")

Please enter the Test Subject as ms_0xx_tpx (example: ms_003_tp1) ms_010_tp1


In [40]:
training_subject_bianca = training_folder
print("The Test Subject is: ", training_subject_bianca)

The Test Subject is:  ms_010_tp1


In [41]:
for file in flairs:
    
    folder_name = "_".join(file.split('_')[0:3])
    
    # For all the images that have a dimension other than the most occuring dimension
    if check_dimensions(file, False) != check_dimensions(ref_img, False):
        print("RESAMPLING....")
        print("FILE TO BE RESAMPLED: ", file)
        print()
                
        # Resampling the Flair Brain Image
        output = run_command(f"cd {directory_1} && flirt -in {file} -ref {ref_img} -out {directory_name}/{folder_name}/{folder_name}_flair_brain.nii.gz -omat {directory_name}/{folder_name}/resampling.mat -interp nearestneighbour -dof 6")
        
        if(folder_name != training_folder):
            
            # FIND THE LESION MASK FOR THE SAME FILE
            for mask in masks:
                if (folder_name in mask):
                    print("Mask to be Resampled: ",mask)
                    mask_file = mask
                    break


            # Resampling the Flair Lesion Mask
            output = run_command(f"cd {directory_1} && flirt -in {mask_file} -ref {ref_img} -out {directory_name}/{folder_name}/{folder_name}_flair_lesion_mask.nii.gz -applyxfm -init {directory_name}/{folder_name}/resampling.mat -interp nearestneighbour -dof 6")

        # Deleting the Resampling.mat file since it is of no use to us   
        output = run_command(f"cd {directory_1} && rm {directory_name}/{folder_name}/resampling.mat")
    
    # In the case that the file does not need to be resampled, we simply just copy/paste it into the specific training directory     
    else:    
        print("COPYING...")
        print("Copying the file:", file)
        # Copying the Flair Brain Images
        output = run_command(f"cd {directory_1} && cp {file} {directory_name}/{folder_name}/{folder_name}_flair_brain.nii.gz")
        
        if(folder_name != training_folder):
            
            # FIND THE LESION MASK FOR THE SAME FILE
            for mask in masks:
                if (folder_name in mask):
                    print("Mask to be Copied: ",mask)
                    mask_file = mask
                    break

            print("Copying the file:", mask_file)
            # Copying the Flair Lesion Mask
            output = run_command(f"cd {directory_1} && cp {mask_file} {directory_name}/{folder_name}/{folder_name}_flair_lesion_mask.nii.gz")
    

COPYING...
Copying the file: ms_010_tp1_flair_brain.nii.gz
COPYING...
Copying the file: ms_003_tp1_flair_brain.nii.gz
Mask to be Copied:  ms_003_tp1_lesion_mask.nii.gz
Copying the file: ms_003_tp1_lesion_mask.nii.gz
COPYING...
Copying the file: ms_009_tp1_flair_brain.nii.gz
Mask to be Copied:  ms_009_tp1_lesion_mask.nii.gz
Copying the file: ms_009_tp1_lesion_mask.nii.gz
COPYING...
Copying the file: ms_004_tp1_flair_brain.nii.gz
Mask to be Copied:  ms_004_tp1_lesion_mask.nii.gz
Copying the file: ms_004_tp1_lesion_mask.nii.gz
COPYING...
Copying the file: ms_007_tp1_flair_brain.nii.gz
Mask to be Copied:  ms_007_tp1_lesion_mask.nii.gz
Copying the file: ms_007_tp1_lesion_mask.nii.gz
COPYING...
Copying the file: ms_001_tp1_flair_brain.nii.gz
Mask to be Copied:  ms_001_tp1_lesion_mask.nii.gz
Copying the file: ms_001_tp1_lesion_mask.nii.gz
COPYING...
Copying the file: ms_006_tp1_flair_brain.nii.gz
Mask to be Copied:  ms_006_tp1_lesion_mask.nii.gz
Copying the file: ms_006_tp1_lesion_mask.nii.gz

## Second Step: Flair Brain to MNI for .mat (Optional)

(This step is an Optional step, and the output is only used when the Flair Brain Images are of very good quality. While we will proceed with the step, we may not be using its results in the training and testing of BIANCA).

Now that we have all the images over the same dimensions, we can move forward with the step of extracting the .mat
file needed to train BIANCA by registering the flair brain image over the sample MNI Image.

#### As a pre-requisite to this step, we need the MNI Space reference file in order to register the resampled flair brains to MNI Space and retrieve the .mat file

<img src="notebook_images/MNI_file.png" alt="Drawing" style="width: 500px;"/>


### Since the location of this file is not fixed, we can go ahead and select the file path for the MNI Space file by running the code block below:

In [42]:
mni_directory = get_directory("file")
print(mni_directory)

/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Datasets/MNI152_2mm_brain.nii.gz


### Then, we actually move forward and apply the registration command to transform the Flair Brain Image to the MNI Space and extract the .mat file

In [43]:
for file in flairs:
    
    folder_name = "_".join(file.split('_')[0:3])
    
    # Registering the FLAIR Brain Image to MNI Space
    print(f"File being Registered is:  {folder_name}_flair_brain.nii.gz")
    output = run_command(f"cd {directory_1}/{directory_name} && flirt -in {folder_name}/{folder_name}_flair_brain.nii.gz -ref {mni_directory} -out {folder_name}/{folder_name}_flair_brain_to_MNI.nii.gz -omat {folder_name}/{folder_name}_flair_brain_to_MNI.mat -bins 256 -cost normcorr -searchrx -180 180 -searchry -180 180 -searchrz -180 180 -dof 7 -interp nearestneighbour")
    print(output)

File being Registered is:  ms_010_tp1_flair_brain.nii.gz

File being Registered is:  ms_003_tp1_flair_brain.nii.gz

File being Registered is:  ms_009_tp1_flair_brain.nii.gz

File being Registered is:  ms_004_tp1_flair_brain.nii.gz

File being Registered is:  ms_007_tp1_flair_brain.nii.gz

File being Registered is:  ms_001_tp1_flair_brain.nii.gz

File being Registered is:  ms_006_tp1_flair_brain.nii.gz

File being Registered is:  ms_005_tp1_flair_brain.nii.gz

File being Registered is:  ms_008_tp1_flair_brain.nii.gz

File being Registered is:  ms_002_tp1_flair_brain.nii.gz



## Third Step: T1 Brain to Flair Brain (Optional)

(As with the Regsitration to MNI Space, this is again an Optional Step and the output is only considered for good quality images)

Following a similar approach to the Second Step, this time we will use the T1 Brain Image and then register it onto the Flair Brain images across all the subjects. These T1 Images (stored in the master directory, will actually be registered to the resampled version of the Flair Brain Images (stored in their respective directories).

As a demonstration, the grey highlighted file ***'ms_003_tp1_T1_brain.nii.gz'*** in the left most column will be registered to the blue highlighted file ***'ms_003_tp1_flair_brain.nii.gz'*** in the right most column.

<img src="notebook_images/t1_flair.png" alt="Drawing" style="width: 1000px;"/>


### Now, we actually move forward and apply the registration command to transform the T1 Brain Image to the Resampled Flair Brain Image and obtain the T1 Brain to Flair Brain Image

In [ ]:
for file in t1s:
    
    folder_name = "_".join(file.split('_')[0:3])
    
    # Registering the T1 Image to Flair Image
    print("Registering the file: ",file)
    output = run_command(f"cd {directory_1} && flirt -in {file} -ref {directory_name}/{folder_name}/{folder_name}_flair_brain.nii.gz -out {directory_name}/{folder_name}/{folder_name}_t1_to_flair.nii.gz -omat {directory_name}/{folder_name}/{folder_name}_t1_to_flair.mat -bins 256 -cost normcorr -searchrx -180 180 -searchry -180 180 -searchrz -180 180 -dof 7 -interp nearestneighbour")
    
    

## So far, we should have the following files and a similar file structure:

<img src="notebook_images/all_file_structure.png" alt="Drawing" style="width: 800px;"/>

## Fourth Step: Generating the Masterfile

Once we have all the files, we will be creating a masterfile.txt which includes all the four files that were highlighted on the previous screenshot, or only the Flair Brain & Lesion Mask, whichever is suitable for us.

For the purposes of our use, we will go ahead with the Flair Brain & Lesion masks only, as follows:

In [44]:
f = open(f"{open_file}/{directory_name}/masterfile.txt", "w")

for file in flairs:
    
    folder_name = "_".join(file.split('_')[0:3])
    
    if(folder_name != training_folder):

        print(f"{folder_name}/{folder_name}_flair_brain.nii.gz {folder_name}/{folder_name}_flair_lesion_mask.nii.gz")

        f.write(f"{folder_name}/{folder_name}_flair_brain.nii.gz {folder_name}/{folder_name}_flair_lesion_mask.nii.gz \n")
        
        # the code below could have been used for making use of the other two files, particularly the MNI Space Registration and the T1 to Flair Registration.
        # f.write(f"{folder_name}/{folder_name}_flair_brain.nii.gz {folder_name}/{folder_name}_t1_to_flair.nii.gz {folder_name}/{folder_name}_flair_brain_to_MNI.mat {folder_name}/{folder_name}_flair_lesion_mask.nii.gz \n")


# Writing the test subject at the very end to correspond with the Training Command (in the upcoming code blocks)
print(f"{training_folder}/{training_folder}_flair_brain.nii.gz {training_folder}/{training_folder}_flair_lesion_mask.nii.gz")

f.write(f"{training_folder}/{training_folder}_flair_brain.nii.gz {training_folder}/{training_folder}_flair_lesion_mask.nii.gz \n")

# the code below could have been used for making use of the other two files, particularly the MNI Space Registration and the T1 to Flair Registration.
# f.write(f"{training_folder}/{training_folder}_flair_brain.nii.gz {training_folder}/{training_folder}_t1_to_flair.nii.gz {training_folder}/{training_folder}_flair_brain_to_MNI.mat {training_folder}/{training_folder}_flair_lesion_mask.nii.gz \n")

        
f.close()

ms_003_tp1/ms_003_tp1_flair_brain.nii.gz ms_003_tp1/ms_003_tp1_flair_lesion_mask.nii.gz
ms_009_tp1/ms_009_tp1_flair_brain.nii.gz ms_009_tp1/ms_009_tp1_flair_lesion_mask.nii.gz
ms_004_tp1/ms_004_tp1_flair_brain.nii.gz ms_004_tp1/ms_004_tp1_flair_lesion_mask.nii.gz
ms_007_tp1/ms_007_tp1_flair_brain.nii.gz ms_007_tp1/ms_007_tp1_flair_lesion_mask.nii.gz
ms_001_tp1/ms_001_tp1_flair_brain.nii.gz ms_001_tp1/ms_001_tp1_flair_lesion_mask.nii.gz
ms_006_tp1/ms_006_tp1_flair_brain.nii.gz ms_006_tp1/ms_006_tp1_flair_lesion_mask.nii.gz
ms_005_tp1/ms_005_tp1_flair_brain.nii.gz ms_005_tp1/ms_005_tp1_flair_lesion_mask.nii.gz
ms_008_tp1/ms_008_tp1_flair_brain.nii.gz ms_008_tp1/ms_008_tp1_flair_lesion_mask.nii.gz
ms_002_tp1/ms_002_tp1_flair_brain.nii.gz ms_002_tp1/ms_002_tp1_flair_lesion_mask.nii.gz
ms_010_tp1/ms_010_tp1_flair_brain.nii.gz ms_010_tp1/ms_010_tp1_flair_lesion_mask.nii.gz


### The file should look similar to the following:

Each subject should have their individual entry on each line

<img src="notebook_images/masterfile_img.png" alt="Drawing" style="width: 1000px;"/>

## Fifth Step: Training BIANCA

Now that we have all the data to Train BIANCA, and it is linked in the Masterfile, we will move forward and train BIANCA

The code block below runs the Training Command for Bianca, and tests it on the very last image. There are numerous other query parameters that could be passed, which are detailed further here:
https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/BIANCA/Userguide#BIANCA_options

In [47]:
training_nums = []
for i in range(len(flairs)):
    training_nums.append(i)

t_nums = ",".join([str(elem+1) for elem in training_nums])
print(t_nums)

print(
    f"cd {directory_1}/{directory_name} && \
    bianca --singlefile=masterfile.txt --trainingnums={t_nums} --labelfeaturenum=2 \
    --querysubjectnum={len(training_nums)} --brainmaskfeaturenum=1 \
    --trainingpts=2000 --nonlespts=10000 --selectpts=noborder -o {training_folder}/bianca_output \
    --saveclassifierdata=mytraining -v")

# The code block commented below could be used for making use of all the 4 files that we generated
# output = run_command(f"cd {directory_1}/{directory_name} && \
#     bianca --singlefile=masterfile.txt --trainingnums={t_nums} --labelfeaturenum=4 \
#     --querysubjectnum={len(training_nums)} --brainmaskfeaturenum=1 --featuresubset=1,2 --matfeaturenum=3 \
#     --trainingpts=2000 --nonlespts=10000 --selectpts=noborder -o {training_folder}/bianca_output \
#     --saveclassifierdata=mytraining -v")

print(output)

1,2,3,4,5,6,7,8,9,10
cd /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02 &&     bianca --singlefile=masterfile.txt --trainingnums=1,2,3,4,5,6,7,8,9,10 --labelfeaturenum=2     --querysubjectnum=10 --brainmaskfeaturenum=1     --trainingpts=2000 --nonlespts=10000 --selectpts=noborder -o ms_010_tp1/bianca_output     --saveclassifierdata=mytraining -v
Number of modalities = 2 , number of possible training subjects = 10
Files are: label file list = , data file list = ['masterfile.txt']
Number of training points = 2000, mode = ['npoints', 'noborder']
no spatial features used
Using all available images as intensity features (no intensity features subset specified)
Loading training data
 ... reading label ms_003_tp1/ms_003_tp1_flair_lesion_mask.nii.gz
 ... reading image ms_003_tp1/ms_003_tp1_flair_brain.nii.gz
 ... reading label ms_009_tp1/ms_009_tp1_flair_lesion_mask.nii.gz
 ... reading image ms_009_tp1/ms_009_tp1_flair_brain.nii.g

## If the above code block ran successfully, we have successfully ran and stored the model

### However, as a final step, we will move ahead and produce a file which contains some basic information, or metadata in proper terminology, from the training so that it could be used when we are testing using this model.

The information we need to store is the reference dimension and the reference image with that dimension

In [133]:
print(most_freq)

('256', '256', '35')


In [165]:
storage_file = open(f"{training_directory}/metadata.txt", "w")
storage_file.write(f"{most_freq[0]},{most_freq[1]},{most_freq[2]}\n")
storage_file.write(f"{ref_img}")

29

# Adaptive Thresholding

This step is needed to build on top of our BIANCA output. The purpose that it would serve is to allow for a dynamic way of applying a threshold, rather than a Binary method of doing so - which would have led to discrepancies in our results.

https://git.fmrib.ox.ac.uk/vaanathi/LOCATE-BIANCA

LOCATE is a supervised method and hence requires data for training. However, since we already have labelled data with which we trained BIANCA, we can simply proceed and process that data further to train LOCATE. 

## Step 1. Creating the Directory & Sub Directories for Training/ Testing

In [55]:
# training_directory_name="Training-2023-02-19" Commented because of kernel stoppage
# open_file = "/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall 2023/Sample_Prepared_Data"
training_directory = open_file + '/' + directory_name
print(training_directory)

/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02


In [57]:
import os
from datetime import date
today = date.today()
directory_name = 'LOCATE_Training-' + str(today)

# Creating the Parent Directory
create_directory(training_directory, directory_name)

locate_training_directory = training_directory + "/"+directory_name

# Creating the Training Sub-Directory
create_directory(locate_training_directory, "Training_imgs_directory")

# Creating the Training Sub-Directory
create_directory(locate_training_directory, "Test_imgs_directory")

# Creating the Sub-Directory for any other processing required
create_directory(locate_training_directory, "Processing")

## Step 2. Preparing the Training Data for LOCATE

In order to obtain results from LOCATE, we need to prepare some files which make use of the data that we already have present, but some additional pre processing on it to generate intermediary files that will be used to train LOCATE.

The required images are obtained in the next few blocks of code.

### 1. The base image modality used in BIANCA
In the format: '(subject_name)-feature-(base_modality_name).nii.gz'

In [65]:
for file in flairs:

    folder_name = "_".join(file.split('_')[0:3])
    
    if(folder_name!=training_subject_bianca):
        training_folder = "Training_imgs_directory"
    else:
        training_folder = "Test_imgs_directory"
    
    folder_name_fixed = "-".join(file.split('_')[0:3])
    
    output = run_command(f"cd {training_directory} && cp {folder_name}/{file} {directory_name}/{training_folder}/{folder_name_fixed}_feature_flair_brain.nii.gz")
    

We will be skipping File No.2 since we are not making use of T1 Images or the MNI Space Registered Images:

_2. Any other additional images that were used as intensity features in BIANCA (--featuresubset) (optional)
<subject_name>_feature_<modality_name>.nii.gz (eg. <subject_name>_feature_T1.nii.gz, please note that it is mandatory to add feature in the filename before the modality name for it to be considered as a feature)_

### 3. Lesion Manual mask (for training subjects only): binary mask (values of 0 and 1) indicating lesion voxels, based on manual segmentation 
As we had manually labelled lesion masks for the training of BIANCA, we will be simply using them in this step:

In the format: (subject_name)_manualmask.nii.gz

In [62]:
training_folder = "Training_imgs_directory"
for mask in masks:

    folder_name = "_".join(mask.split('_')[0:3])
    folder_name_fixed = "-".join(mask.split('_')[0:3])
    
    if(folder_name!=training_subject_bianca):
        output = run_command(f"cd {training_directory} && cp {folder_name}/{folder_name}_flair_lesion_mask.nii.gz {directory_name}/{training_folder}/{folder_name_fixed}_manualmask.nii.gz")
        

### The following files can be obtained together:

### 4. Ventricle distance map: image where each voxel intensity represents the distance from ventricles within the brain mask

### 6. Brain mask: binary mask obtained from FSL-BET or any other method

### 7. BIANCA mask: binary mask, as obtained from make_bianca_mask (white matter mask excluding sub-cortical regions) - If you are not using BIANCA mask in your analysis, make a copy of the brain mask and rename it as BIANCA mask.

In the formats:
- (subject_name)_ventdistmap.nii.gz
- (subject_name)_brainmask.nii.gz
- (subject_name)_biancamask.nii.gz

With the ultimate goal to obtain the ventdistmap, we will be obtaining the brain and bianca masks as intermediary files. The ventdistmap can be calculated using the FSL tool distancemap – see the wiki for more details. Example call: 

distancemap -i
<ventricle_mask_image_in_FLAIR_space> -m <brain_mask_in_FLAIR_space> -o
<subject_name>_ventdistmap.nii.gz
 <subject_name>_ventdistmap.nii.gz

#### For this, we first run FSL Fast to get the CSF pve file

In [64]:
for file in flairs:
    
    folder_name = "_".join(file.split('_')[0:3])
    
    if(folder_name!=training_subject_bianca):
        training_folder = "Training_imgs_directory"
    else:
        training_folder = "Test_imgs_directory"
    
    folder_name_fixed = "-".join(file.split('_')[0:3])
    
    print(f"cd {training_directory} && /usr/local/fsl/bin/fast -t 1 -n 3 -H 0.1 -I 4 -l 20.0 -o {directory_name}/Processing {directory_name}/{training_folder}/{folder_name_fixed}_feature_FLAIR.nii.gz")
    print()
    print(f"cd {training_directory} && cp {directory_name}/Processing/_pve_0.nii.gz {directory_name}/{training_folder}/{folder_name_fixed}_csf_pve.nii.gz")
    print()
    
    output = run_command(f"cd {training_directory} && /usr/local/fsl/bin/fast -t 1 -n 3 -H 0.1 -I 4 -l 20.0 -o {directory_name}/Processing/ {directory_name}/{training_folder}/{folder_name_fixed}_feature_FLAIR.nii.gz")
    print(output)  
    
    output = run_command(f"cd {training_directory} && cp {directory_name}/Processing/_pve_0.nii.gz {directory_name}/{training_folder}/{folder_name_fixed}_csf_pve.nii.gz")
    print(output) 
    
        


cd /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02 && /usr/local/fsl/bin/fast -t 1 -n 3 -H 0.1 -I 4 -l 20.0 -o LOCATE_Training-2023-04-02/Processing LOCATE_Training-2023-04-02/Test_imgs_directory/ms-010-tp1_feature_FLAIR.nii.gz

cd /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02 && cp LOCATE_Training-2023-04-02/Processing/_pve_0.nii.gz LOCATE_Training-2023-04-02/Test_imgs_directory/ms-010-tp1_csf_pve.nii.gz



cd /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02 && /usr/local/fsl/bin/fast -t 1 -n 3 -H 0.1 -I 4 -l 20.0 -o LOCATE_Training-2023-04-02/Processing LOCATE_Training-2023-04-02/Training_imgs_directory/ms-003-tp1_feature_FLAIR.nii.gz

cd /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02 && cp LOCATE_Training-2023-04-02/Processing/_p

#### Here, we need the T1 to MNI Space Image. I have referenced that here, however you can uncomment the get_directory command and select the file!

In [78]:
# Obtain the T1_@_MNI152_2mm.cnf
# t1_2_mni = get_directory("folder")
t1_2_mni = "/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Datasets/T1_2_MNI152_2mm.cnf"


/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Datasets/MNI152_2mm_brain.nii.gz
/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Datasets/T1_2_MNI152_2mm.cnf


#### Next, obtain the warp_file_MNI2structural

- First Run FLIRT to obtain the affine_transf.mat file
- Use that with FNIRT to obtain the nonlineaer_transf file
- Use the nonlineaer_transf and calculate its inverse to get the warp_file_MNI2structural
- Copy the output file into the respective folder

In [81]:
for file in flairs:
    
    folder_name = "_".join(file.split('_')[0:3])
    if(folder_name!=training_subject_bianca):
        training_folder = "Training_imgs_directory"
    else:
        training_folder = "Test_imgs_directory"
    folder_name_fixed = "-".join(file.split('_')[0:3])
    
    print(f"cd {training_directory} && flirt -ref {mni_directory} -in {directory_name}/{training_folder}/{folder_name_fixed}_feature_flair_brain.nii.gz -omat {directory_name}/Processing/my_affine_transf.mat")
    print(f"cd {training_directory} && fnirt --in={directory_name}/{training_folder}/{folder_name_fixed}_feature_flair_brain.nii.gz --aff={directory_name}/Processing/my_affine_transf.mat --cout={directory_name}/Processing/my_nonlinear_transf --config={t1_2_mni} --lambda=400,200,150,75,60,45")
    print(f"cd {training_directory} && invwarp -w {directory_name}/Processing/my_nonlinear_transf -o {directory_name}/Processing/invwarpvol_new -r {directory_name}/{training_folder}/{folder_name_fixed}_feature_FLAIR.nii.gz")
    print(f"cd {training_directory} && cp {directory_name}/Processing/invwarpvol_new.nii.gz {directory_name}/{training_folder}/{folder_name_fixed}_warp_file_MNI2structural.nii.gz")

    output = run_command(f"cd {training_directory} && flirt -ref {mni_directory} -in {directory_name}/{training_folder}/{folder_name_fixed}_feature_flair_brain.nii.gz -omat {directory_name}/Processing/my_affine_transf.mat")
    print(output) 
    output = run_command(f"cd {training_directory} && fnirt --in={directory_name}/{training_folder}/{folder_name_fixed}_feature_flair_brain.nii.gz --aff={directory_name}/Processing/my_affine_transf.mat --cout={directory_name}/Processing/my_nonlinear_transf --config={t1_2_mni} --lambda=400,200,150,75,60,45")
    print(output) 
    output = run_command(f"cd {training_directory} && invwarp -w {directory_name}/Processing/my_nonlinear_transf -o {directory_name}/Processing/invwarpvol_new -r {directory_name}/{training_folder}/{folder_name_fixed}_feature_flair_brain.nii.gz")
    print(output) 
    output = run_command(f"cd {training_directory} && cp {directory_name}/Processing/invwarpvol_new.nii.gz {directory_name}/{training_folder}/{folder_name_fixed}_warp_file_MNI2structural.nii.gz")
    print(output) 

    print()
    print()


cd /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02 && flirt -ref /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Datasets/MNI152_2mm_brain.nii.gz -in LOCATE_Training-2023-04-02/Test_imgs_directory/ms-010-tp1_feature_flair_brain.nii.gz -omat LOCATE_Training-2023-04-02/Processing/my_affine_transf.mat
cd /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02 && fnirt --in=LOCATE_Training-2023-04-02/Test_imgs_directory/ms-010-tp1_feature_flair_brain.nii.gz --aff=LOCATE_Training-2023-04-02/Processing/my_affine_transf.mat --cout=LOCATE_Training-2023-04-02/Processing/my_nonlinear_transf --config=/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Datasets/T1_2_MNI152_2mm.cnf --lambda=400,200,150,75,60,45
cd /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02 && invwarp -w L









cd /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02 && flirt -ref /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Datasets/MNI152_2mm_brain.nii.gz -in LOCATE_Training-2023-04-02/Training_imgs_directory/ms-006-tp1_feature_flair_brain.nii.gz -omat LOCATE_Training-2023-04-02/Processing/my_affine_transf.mat
cd /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02 && fnirt --in=LOCATE_Training-2023-04-02/Training_imgs_directory/ms-006-tp1_feature_flair_brain.nii.gz --aff=LOCATE_Training-2023-04-02/Processing/my_affine_transf.mat --cout=LOCATE_Training-2023-04-02/Processing/my_nonlinear_transf --config=/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Datasets/T1_2_MNI152_2mm.cnf --lambda=400,200,150,75,60,45
cd /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02

#### Then we simply run the make_bianca_mask and the distancemap commands

In [100]:
for file in flairs:
    
    folder_name = "_".join(file.split('_')[0:3])
    if(folder_name!=training_subject_bianca):
        training_folder = "Training_imgs_directory"
    else:
        training_folder = "Test_imgs_directory"
    folder_name_fixed = "-".join(file.split('_')[0:3])
    
    print(f"cd {training_directory} && make_bianca_mask {directory_name}/{training_folder}/{folder_name_fixed}_feature_flair.nii.gz {directory_name}/{training_folder}/{folder_name_fixed}_csf_pve.nii.gz {directory_name}/{training_folder}/{folder_name_fixed}_warp_file_MNI2structural.nii.gz 0")
    # make_bianca_mask ms-009-tp1_feature_FLAIR.nii.gz ms-009-tp1_csf_pve.nii.gz ms-009-tp1_warp_file_MNI2structural.nii.gz 0
    
    print(f"cd {training_directory} && distancemap -i {directory_name}/{training_folder}/{folder_name_fixed}_feature_flair_ventmask.nii.gz -m {directory_name}/{training_folder}/{folder_name_fixed}_feature_flair_brain_mask.nii.gz -o {directory_name}/{training_folder}/{folder_name_fixed}_ventdistmap.nii.gz")
    # distancemap -i ms-009-tp1_feature_FLAIR_ventmask.nii.gz -m ms-009-tp1_brainmask.nii.gz -o ms-009-tp1_ventdistmap.nii.gz

    output = run_command(f"cd {training_directory} && make_bianca_mask {directory_name}/{training_folder}/{folder_name_fixed}_feature_flair.nii.gz {directory_name}/{training_folder}/{folder_name_fixed}_csf_pve.nii.gz {directory_name}/{training_folder}/{folder_name_fixed}_warp_file_MNI2structural.nii.gz 0")
    print(output)
    
    output = run_command(f"cd {training_directory} && distancemap -i {directory_name}/{training_folder}/{folder_name_fixed}_feature_flair_ventmask.nii.gz -m {directory_name}/{training_folder}/{folder_name_fixed}_feature_flair_brain_mask.nii.gz -o {directory_name}/{training_folder}/{folder_name_fixed}_ventdistmap.nii.gz")
    print(output)
    

cd /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02 && make_bianca_mask LOCATE_Training-2023-04-02/Test_imgs_directory/ms-010-tp1_feature_flair.nii.gz LOCATE_Training-2023-04-02/Test_imgs_directory/ms-010-tp1_csf_pve.nii.gz LOCATE_Training-2023-04-02/Test_imgs_directory/ms-010-tp1_warp_file_MNI2structural.nii.gz 0
cd /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02 && distancemap -i LOCATE_Training-2023-04-02/Test_imgs_directory/ms-010-tp1_feature_flair_ventmask.nii.gz -m LOCATE_Training-2023-04-02/Test_imgs_directory/ms-010-tp1_feature_flair_brain_mask.nii.gz -o LOCATE_Training-2023-04-02/Test_imgs_directory/ms-010-tp1_ventdistmap.nii.gz
generating ms-010-tp1_feature_flair_brain_mask.nii.gz
creating ventricles mask
creating WM mask


cd /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02 && make_bianca

#### Data Cleaning - We rename and move around some files to make it organized and match our naming conventions in line with LOCATE

In [102]:
for file in flairs:
    
    folder_name = "_".join(file.split('_')[0:3])
    
    if(folder_name!=training_subject_bianca):
        training_folder = "Training_imgs_directory"
    else:
        training_folder = "Test_imgs_directory"
    
    folder_name_fixed = "-".join(file.split('_')[0:3])

    print(f"cd {training_directory} && mv {directory_name}/{training_folder}/{folder_name_fixed}_feature_flair_brain_mask.nii.gz {directory_name}/{training_folder}/{folder_name_fixed}_brainmask.nii.gz")
    print(f"cd {training_directory} && mv {directory_name}/{training_folder}/{folder_name_fixed}_feature_flair_bianca_mask.nii.gz {directory_name}/{training_folder}/{folder_name_fixed}_biancamask.nii.gz")
    print(f"cd {training_directory} && mv {directory_name}/{training_folder}/{folder_name_fixed}_feature_flair_brain.nii.gz {directory_name}/{training_folder}/{folder_name_fixed}_feature_FLAIR.nii.gz")
    print(f"cd {training_directory} && rm {directory_name}/{training_folder}/{folder_name_fixed}_feature_flair_brain_to_MNI152_T1_2mm.log")    
    print(f"cd {training_directory} && rm {directory_name}/{training_folder}/{folder_name_fixed}_feature_flair_ventmask.nii.gz")    
    print(f"cd {training_directory} && rm {directory_name}/{training_folder}/{folder_name_fixed}_csf_pve.nii.gz")    
    print(f"cd {training_directory} && rm {directory_name}/{training_folder}/{folder_name_fixed}_warp_file_MNI2structural.nii.gz")    
    
    
    output = run_command(f"cd {training_directory} && mv {directory_name}/{training_folder}/{folder_name_fixed}_feature_flair_brain_mask.nii.gz {directory_name}/{training_folder}/{folder_name_fixed}_brainmask.nii.gz")
    print(output)
    
    output = run_command(f"cd {training_directory} && mv {directory_name}/{training_folder}/{folder_name_fixed}_feature_flair_bianca_mask.nii.gz {directory_name}/{training_folder}/{folder_name_fixed}_biancamask.nii.gz")
    print(output)

    output = run_command(f"cd {training_directory} && mv {directory_name}/{training_folder}/{folder_name_fixed}_feature_flair_brain.nii.gz {directory_name}/{training_folder}/{folder_name_fixed}_feature_FLAIR.nii.gz")
    print(output)
    
    output = run_command(f"cd {training_directory} && rm {directory_name}/{training_folder}/{folder_name_fixed}_feature_flair_brain_to_MNI152_T1_2mm.log")
    print(output)
    
    output = run_command(f"cd {training_directory} && rm {directory_name}/{training_folder}/{folder_name_fixed}_feature_flair_ventmask.nii.gz")
    print(output)
    
    output = run_command(f"cd {training_directory} && rm {directory_name}/{training_folder}/{folder_name_fixed}_csf_pve.nii.gz")
    print(output)
    
    output = run_command(f"cd {training_directory} && rm {directory_name}/{training_folder}/{folder_name_fixed}_warp_file_MNI2structural.nii.gz")
    print(output)

    

cd /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02 && mv LOCATE_Training-2023-04-02/Test_imgs_directory/ms-010-tp1_feature_flair_brain_mask.nii.gz LOCATE_Training-2023-04-02/Test_imgs_directory/ms-010-tp1_brainmask.nii.gz
cd /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02 && mv LOCATE_Training-2023-04-02/Test_imgs_directory/ms-010-tp1_feature_flair_bianca_mask.nii.gz LOCATE_Training-2023-04-02/Test_imgs_directory/ms-010-tp1_biancamask.nii.gz
cd /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02 && mv LOCATE_Training-2023-04-02/Test_imgs_directory/ms-010-tp1_feature_flair_brain.nii.gz LOCATE_Training-2023-04-02/Test_imgs_directory/ms-010-tp1_feature_FLAIR.nii.gz
cd /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02 && rm LOCATE_Training-2023



cd /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02 && mv LOCATE_Training-2023-04-02/Training_imgs_directory/ms-006-tp1_feature_flair_brain_mask.nii.gz LOCATE_Training-2023-04-02/Training_imgs_directory/ms-006-tp1_brainmask.nii.gz
cd /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02 && mv LOCATE_Training-2023-04-02/Training_imgs_directory/ms-006-tp1_feature_flair_bianca_mask.nii.gz LOCATE_Training-2023-04-02/Training_imgs_directory/ms-006-tp1_biancamask.nii.gz
cd /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02 && mv LOCATE_Training-2023-04-02/Training_imgs_directory/ms-006-tp1_feature_flair_brain.nii.gz LOCATE_Training-2023-04-02/Training_imgs_directory/ms-006-tp1_feature_FLAIR.nii.gz
cd /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02 



### 5. BIANCA output: unthresholded lesion probability map (LPM) obtained from BIANCA
In the format:
- (subject_name)_BIANCA_LPM.nii.gz

From the above training of BIANCA, we obtained some training files which we are going to use to generate bianca outputs for all of the files that we used for the training

#### As we did previously, we are going to use a similar approach to generate the BIANCA Outputs

In [112]:
file_path = (f"{training_directory}").replace(" ", "\ ")
classifier = file_path + "/mytraining"
print("Classifier: ",classifier)
print()

for file in flairs:
    
    folder_name = "-".join(file.split('_')[0:3])

    masterfile = f"{training_directory}/{directory_name}/Processing/{folder_name}_masterfile.txt"
    
    print(masterfile)

    print(f"{directory_name}/Training_imgs_directory/{folder_name}_feature_FLAIR.nii.gz")
    print()
    file = open(masterfile, 'w')
    file.write(f"{directory_name}/Training_imgs_directory/{folder_name}_feature_FLAIR.nii.gz")
    file.close()
    
    masterfile=masterfile.replace(" ", "\ ")
    bianca_output = f"{folder_name}_BIANCA_LPM.nii.gz"
    
    print(f"cd {file_path} && bianca --singlefile={masterfile} --loadclassifierdata={classifier} --querysubjectnum=1 --brainmaskfeaturenum=1 -o {directory_name}/Training_imgs_directory/{bianca_output} -v")
    print()
    
    p = os.popen(
        f"cd {file_path} && bianca --singlefile={masterfile} --loadclassifierdata={classifier} --querysubjectnum=1 --brainmaskfeaturenum=1 -o {directory_name}/Training_imgs_directory/{bianca_output} -v")
    if p:
        output = p.read()
        print(output)
    

Classifier:  /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02/mytraining

/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02/LOCATE_Training-2023-04-02/Processing/ms-010-tp1_masterfile.txt
LOCATE_Training-2023-04-02/Training_imgs_directory/ms-010-tp1_feature_FLAIR.nii.gz

cd /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02 && bianca --singlefile=/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02/LOCATE_Training-2023-04-02/Processing/ms-010-tp1_masterfile.txt --loadclassifierdata=/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02/mytraining --querysubjectnum=1 --brainmaskfeaturenum=1 -o LOCATE_Training-2023-04-02/Training_imgs_directory/ms-010-tp1_BIANCA_LPM.nii.gz -v

Number of moda

Number of modalities = 1 , number of possible training subjects = 1
Files are: label file list = , data file list = ['/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02/LOCATE_Training-2023-04-02/Processing/ms-001-tp1_masterfile.txt']
Number of training points = 2000, mode = []
no spatial features used
Using all available images as intensity features (no intensity features subset specified)
Loading training data  /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02/mytraining
Training data is size (108000, 1)
Loading query data
  Query file list = ['LOCATE_Training-2023-04-02/Training_imgs_directory/ms-001-tp1_feature_FLAIR.nii.gz']
 ... reading image LOCATE_Training-2023-04-02/Training_imgs_directory/ms-001-tp1_feature_FLAIR.nii.gz
Training classifier
Applying classifier
Calculating p-values
Saving output image

/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Ana

#### Same thing, but for the Testing Files now

In [114]:
file = f"{training_subject_bianca}_tp1_flair_brain.nii.gz"

folder_name = "-".join(file.split('_')[0:3])

masterfile = f"{training_directory}/{directory_name}/Processing/{folder_name}_masterfile.txt"
print(file)
print()
print()
print(f"{directory_name}/Test_imgs_directory/{folder_name}_feature_FLAIR.nii.gz")
print()
file = open(masterfile, 'w')
file.write(f"{directory_name}/Test_imgs_directory/{folder_name}_feature_FLAIR.nii.gz")
file.close()

masterfile=masterfile.replace(" ", "\ ")
bianca_output = f"{folder_name}_BIANCA_LPM.nii.gz"

print(f"cd {file_path} && bianca --singlefile={masterfile} --loadclassifierdata={classifier} --querysubjectnum=1 --brainmaskfeaturenum=1 -o {directory_name}/Test_imgs_directory/{bianca_output} -v")
print()

p = os.popen(
    f"cd {file_path} && bianca --singlefile={masterfile} --loadclassifierdata={classifier} --querysubjectnum=1 --brainmaskfeaturenum=1 -o {directory_name}/Test_imgs_directory/{bianca_output} -v")
if p:x
    output = p.read()
    print(output)

ms_010_tp1_tp1_flair_brain.nii.gz


LOCATE_Training-2023-04-02/Test_imgs_directory/ms-010-tp1_feature_FLAIR.nii.gz

cd /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02 && bianca --singlefile=/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02/LOCATE_Training-2023-04-02/Processing/ms-010-tp1_masterfile.txt --loadclassifierdata=/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02/mytraining --querysubjectnum=1 --brainmaskfeaturenum=1 -o LOCATE_Training-2023-04-02/Test_imgs_directory/ms-010-tp1_BIANCA_LPM.nii.gz -v

Number of modalities = 1 , number of possible training subjects = 1
Files are: label file list = , data file list = ['/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02/LOCATE_Training-2023-04-02/Processing/ms-010-tp1_masterfile.txt']
Num

## Step 3: Running the LOCATE Training

In order to run the LOCATE Training File, we have to have MATLAB installed as a pre requisite. 

Then, we have to call the directory of the MATLAB, which, in our case, is '/Applications/MATLAB_R2022b.app/bin/matlab'. Using this, we have to call the locate training function, and pass it the trainings_imgs_directory as a parameter (we generated this directory and populated this in Step 2)

In [130]:
print(f"/Applications/MATLAB_R2022b.app/bin/matlab -nodisplay -r \"cd('{locate_training_directory}'); LOCATE_training('Training_imgs_directory');exit\"")

output = run_command(f"/Applications/MATLAB_R2022b.app/bin/matlab -nodisplay -r \"cd('{locate_training_directory}'); LOCATE_training('Training_imgs_directory');exit\"")


/Applications/MATLAB_R2022b.app/bin/matlab -nodisplay -r "cd('/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02/LOCATE_Training-2023-04-02'); LOCATE_training('Training_imgs_directory');exit"


## Step 4: LOCATE Testing

Once we have run the LOCATE Training file, we will move ahead with testing the regression model stored under LOCATE_Training_files. This will allow us to utilize this model on our prepared sets of images!

In [131]:
print(f"/Applications/MATLAB_R2022b.app/bin/matlab -nodisplay -r \"cd('{locate_training_directory}'); LOCATE_testing('Test_imgs_directory', 'Training_imgs_directory');exit\"")

output = run_command(f"/Applications/MATLAB_R2022b.app/bin/matlab -nodisplay -r \"cd('{locate_training_directory}'); LOCATE_testing('Test_imgs_directory', 'Training_imgs_directory');exit\"")


/Applications/MATLAB_R2022b.app/bin/matlab -nodisplay -r "cd('/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02/LOCATE_Training-2023-04-02'); LOCATE_testing('Test_imgs_directory', 'Training_imgs_directory');exit"


Modifying the MetaData from our Training to add the details of the most recent LOCATE Training Directory

In [198]:
storage_file.close()
storage_file = open(f"{training_directory}/metadata.txt", "a")
storage_file.write(f"\n{locate_training_directory}\n")
storage_file.close()

## Step 5: Storing Model for the Graphical User Interface

Though there is a section of this notebook catering to the Longitduinal Analysis, we also have a Graphical User Interface which utilizes the produced models. Therefore, before we move forward with that, we will be updating/ storing the model at the appropriate location (which is the '/GUI/assets/model' directory.

Moreover, when the Model, we have a lot of intermediary files that were generated which we are not going to be utilising so we will not be copying them into the 'model' directory for the GUI

In [34]:
import os
from pathlib import Path
current_directory = os.getcwd()
path = Path(locate_training_directory)
locate_parent_directory = path.parent.absolute()


output = run_command(f"cp -r {locate_training_directory}/Training_imgs_directory {current_directory}/GUI/assets/model")
print(output)

output = run_command(f"cp {locate_parent_directory}/masterfile.txt {current_directory}/GUI/assets/model/masterfile.txt")
print(output)

output = run_command(f"cp {locate_parent_directory}/metadata.txt {current_directory}/GUI/assets/model/metadata.txt")
print(output)

output = run_command(f"cp {locate_parent_directory}/mytraining {current_directory}/GUI/assets/model/mytraining")
print(output)

output = run_command(f"cp {locate_parent_directory}/mytraining_labels {current_directory}/GUI/assets/model/mytraining_labels")
print(output)


# Longitudinal Analysis

Now comes the primary part of this project, the Longitudinal Analysis. 

So far, we have a trained model for BIANCA, and also a trained model for LOCATE. We will go ahead and utilize those for a Longitudinal Analysis!

### Get the LOCATE Training Directory 
This is the directory where our trained model is stored, which we will be using to obtain the segmentations. If we are running the code right after the model training, we should have our locate training directory stored. Otherwise we can simply obtain it with the get_directory command. This is to facilitate any other previously trained models as well!

In [35]:
# locate_training_directory = '/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02/LOCATE_Training-2023-04-02'
# locate_training_directory = get_directory("folder")
print(locate_training_directory)


/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02/LOCATE_Training-2023-04-02


### Create a directory within Locate Training for Processing the Longitudinal Analysis files

In [5]:
import os
create_directory(locate_training_directory, "Longitudinal_Processing")
create_directory(locate_training_directory, "Longitudinal_Tests")
processing_directory = locate_training_directory+"/Longitudinal_Processing"
testing_directory = locate_training_directory+"/Longitudinal_Tests"
print(processing_directory)


[Errno 17] File exists: '/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02/LOCATE_Training-2023-04-02/Longitudinal_Tests'
/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02/LOCATE_Training-2023-04-02/Longitudinal_Processing


## Step 1: Taking User Input for the 2 different Timepoints

For our analysis, we need 2 user input timepoints which we also ask for in our Graphical User Interface

### Getting the First Timepoint

In [7]:
first_timepoint = get_directory("file")
print(first_timepoint)

/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Longitudinal_Input/2_Sag_Timepoints/FLAIR_Sag_3mm_20180609132051_2.nii.gz


### Getting the Second Timepoint

In [8]:
second_timepoint = get_directory("file")
print(second_timepoint)

/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Longitudinal_Input/2_Sag_Timepoints/FLAIR_Sag_3mm_20210907085543_2.nii.gz


## Step 2: Brain Extraction

Getting Parent Directory of where the timepoints are stored

In [20]:
from pathlib import Path
path = Path(second_timepoint)
timepoints_directory = path.parent.absolute()
print(timepoints_directory)

/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Longitudinal_Input/2_Sag_Timepoints


### Extraction

In [30]:
output = run_command(f"cd {timepoints_directory} && bet {first_timepoint} {processing_directory}/tp1_brain.nii.gz")
output = run_command(f"cd {timepoints_directory} && bet {second_timepoint} {processing_directory}/tp2_feature_FLAIR.nii.gz")


## Step 3: Register the Timepoints

Since the two timepoints input by the user are very likely to be of different dimensions, and also in different spaces, we will be resampling the First Timepoint onto the Second Timepoint.

### Resampling:

In [31]:
print(f"cd {processing_directory} && flirt -in tp1_brain.nii.gz -ref tp2_feature_FLAIR.nii.gz -out tp1_feature_FLAIR.nii.gz -omat tp1_feature_FLAIR.mat -interp nearestneighbour -dof 6")
output = run_command(f"cd {processing_directory} && flirt -in tp1_brain.nii.gz -ref tp2_feature_FLAIR.nii.gz -out tp1_feature_FLAIR.nii.gz -omat tp1_feature_FLAIR.mat -interp nearestneighbour -dof 6")


cd /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02/LOCATE_Training-2023-04-02/Longitudinal_Processing && flirt -in tp1_brain.nii.gz -ref tp2_feature_FLAIR.nii.gz -out tp1_feature_FLAIR.nii.gz -omat tp1_feature_FLAIR.mat -interp nearestneighbour -dof 6


## Step 4: Generate Intermediary Files

Once we have the registered and resampled timepoints, we will go ahead and produce the files that we did for the Testing Data previously for LOCATE. This includes generating the following files:

- Brain Mask
- BIANCA Mask
- Ventricle Distance Map
- BIANCA's Output of a Lesion Probability Map

### Obtaining all mentioned files for Timepoint 1

In [32]:
output = run_command(f"cd {locate_training_directory} && /usr/local/fsl/bin/fast -t 1 -n 3 -H 0.1 -I 4 -l 20.0 -o {locate_training_directory}/Longitudinal_Processing/ {locate_training_directory}/Longitudinal_Processing/tp1_feature_FLAIR.nii.gz")

output = run_command(f"cd {locate_training_directory} && cp {locate_training_directory}/Longitudinal_Processing/_pve_0.nii.gz {locate_training_directory}/Longitudinal_Processing/tp1_csf_pve.nii.gz")

output = run_command(f"cd {locate_training_directory} && /usr/local/fsl/bin/fast -t 1 -n 3 -H 0.1 -I 4 -l 20.0 -o {locate_training_directory}/Longitudinal_Processing/ {locate_training_directory}/Longitudinal_Processing/tp2_feature_FLAIR.nii.gz")

output = run_command(f"cd {locate_training_directory} && cp {locate_training_directory}/Longitudinal_Processing/_pve_0.nii.gz {locate_training_directory}/Longitudinal_Processing/tp2_csf_pve.nii.gz")

t1_2_mni = "/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Datasets/T1_2_MNI152_2mm.cnf"
mni_directory = "/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Datasets/MNI152_2mm_brain.nii.gz"

output = run_command(f"cd {locate_training_directory} && flirt -ref {mni_directory} -in {locate_training_directory}/Longitudinal_Processing/tp1_feature_FLAIR.nii.gz -omat {locate_training_directory}/Longitudinal_Processing/my_affine_transf.mat") 

output = run_command(f"cd {locate_training_directory} && fnirt --in={locate_training_directory}/Longitudinal_Processing/tp1_feature_FLAIR.nii.gz --aff={locate_training_directory}/Longitudinal_Processing/my_affine_transf.mat --cout={locate_training_directory}/Longitudinal_Processing/my_nonlinear_transf --config={t1_2_mni} --lambda=400,200,150,75,60,45")

output = run_command(f"cd {locate_training_directory} && invwarp -w {locate_training_directory}/Longitudinal_Processing/my_nonlinear_transf -o {locate_training_directory}/Longitudinal_Processing/invwarpvol_new -r {locate_training_directory}/Longitudinal_Processing/tp1_feature_FLAIR.nii.gz") 

output = run_command(f"cd {locate_training_directory} && cp {locate_training_directory}/Longitudinal_Processing/invwarpvol_new.nii.gz {locate_training_directory}/Longitudinal_Processing/tp1_warp_file_MNI2structural.nii.gz")

output = run_command(f"cd {locate_training_directory} && cp {locate_training_directory}/Longitudinal_Processing/tp1_feature_FLAIR.nii.gz {locate_training_directory}/Longitudinal_Processing/tp1_feature_FLAIR_brain.nii.gz")

output = run_command(f"cd {locate_training_directory} && make_bianca_mask {locate_training_directory}/Longitudinal_Processing/tp1_feature_FLAIR.nii.gz {locate_training_directory}/Longitudinal_Processing/tp1_csf_pve.nii.gz {locate_training_directory}/Longitudinal_Processing/tp1_warp_file_MNI2structural.nii.gz 0")

output = run_command(f"cd {locate_training_directory} && distancemap -i {locate_training_directory}/Longitudinal_Processing/tp1_feature_flair_ventmask.nii.gz -m {locate_training_directory}/Longitudinal_Processing/tp1_feature_FLAIR_brain_mask.nii.gz -o {locate_training_directory}/Longitudinal_Processing/tp1_ventdistmap.nii.gz")

path = Path(locate_training_directory)
locate_parent_directory = path.parent.absolute()


masterfile = f"{locate_training_directory}/Longitudinal_Processing/tp1_masterfile.txt"
print(f"{locate_training_directory}/Longitudinal_rocessing/tp1_feature_FLAIR.nii.gz")
print()
file = open(masterfile, 'w')
file.write(f"{locate_training_directory}/Longitudinal_Processing/tp1_feature_FLAIR.nii.gz")
file.close()

masterfile=masterfile.replace(" ", "\ ")
bianca_output = "tp1_BIANCA_LPM.nii.gz"

print(f"cd {locate_parent_directory} && bianca --singlefile={masterfile} --loadclassifierdata={locate_parent_directory}/mytraining --querysubjectnum=1 --brainmaskfeaturenum=1 -o {locate_training_directory}/Longitudinal_Processing/tp1_BIANCA_LPM.nii.gz -v")
print()

p = os.popen(
    f"cd {locate_parent_directory} && bianca --singlefile={masterfile} --loadclassifierdata={locate_parent_directory}/mytraining --querysubjectnum=1 --brainmaskfeaturenum=1 -o {locate_training_directory}/Longitudinal_Processing/tp1_BIANCA_LPM.nii.gz -v")
if p:
    output = p.read()
    print(output)
    
# Storing them appropriately
output = run_command(f"cd {locate_training_directory} && mv {locate_training_directory}/Longitudinal_Processing/tp1_feature_FLAIR.nii.gz {testing_directory}/tp1_feature_FLAIR.nii.gz")

output = run_command(f"cd {locate_training_directory} && mv {locate_training_directory}/Longitudinal_Processing/tp1_feature_FLAIR_bianca_mask.nii.gz {testing_directory}/tp1_biancamask.nii.gz")

output = run_command(f"cd {locate_training_directory} && mv {locate_training_directory}/Longitudinal_Processing/tp1_feature_FLAIR_brain_mask.nii.gz {testing_directory}/tp1_brainmask.nii.gz")

output = run_command(f"cd {locate_training_directory} && mv {locate_training_directory}/Longitudinal_Processing/tp1_ventdistmap.nii.gz {testing_directory}/tp1_ventdistmap.nii.gz")

output = run_command(f"cd {locate_training_directory} && mv {locate_training_directory}/Longitudinal_Processing/tp1_BIANCA_LPM.nii.gz {testing_directory}/tp1_BIANCA_LPM.nii.gz")


### Obtaining all mentioned files for Timepoint 2

In [52]:
output = run_command(f"cd {locate_training_directory} && flirt -ref {mni_directory} -in {locate_training_directory}/Longitudinal_Processing/tp2_feature_FLAIR.nii.gz -omat {locate_training_directory}/Longitudinal_Processing/my_affine_transf.mat") 

output = run_command(f"cd {locate_training_directory} && fnirt --in={locate_training_directory}/Longitudinal_Processing/tp2_feature_FLAIR.nii.gz --aff={locate_training_directory}/Longitudinal_Processing/my_affine_transf.mat --cout={locate_training_directory}/Longitudinal_Processing/my_nonlinear_transf --config={t1_2_mni} --lambda=400,200,150,75,60,45")

output = run_command(f"cd {locate_training_directory} && invwarp -w {locate_training_directory}/Longitudinal_Processing/my_nonlinear_transf -o {locate_training_directory}/Longitudinal_Processing/invwarpvol_new -r {locate_training_directory}/Longitudinal_Processing/tp2_feature_FLAIR.nii.gz") 

output = run_command(f"cd {locate_training_directory} && cp {locate_training_directory}/Longitudinal_Processing/invwarpvol_new.nii.gz {locate_training_directory}/Longitudinal_Processing/tp2_warp_file_MNI2structural.nii.gz")

output = run_command(f"cd {locate_training_directory} && cp {locate_training_directory}/Longitudinal_Processing/tp2_feature_FLAIR.nii.gz {locate_training_directory}/Longitudinal_Processing/tp2_feature_FLAIR_brain.nii.gz")

output = run_command(f"cd {locate_training_directory} && make_bianca_mask {locate_training_directory}/Longitudinal_Processing/tp2_feature_FLAIR.nii.gz {locate_training_directory}/Longitudinal_Processing/tp2_csf_pve.nii.gz {locate_training_directory}/Longitudinal_Processing/tp2_warp_file_MNI2structural.nii.gz 0")

output = run_command(f"cd {locate_training_directory} && distancemap -i {locate_training_directory}/Longitudinal_Processing/tp2_feature_flair_ventmask.nii.gz -m {locate_training_directory}/Longitudinal_Processing/tp2_feature_FLAIR_brain_mask.nii.gz -o {locate_training_directory}/Longitudinal_Processing/tp2_ventdistmap.nii.gz")

masterfile = f"{locate_training_directory}/Longitudinal_Processing/tp2_masterfile.txt"
print(f"{locate_training_directory}/Longitudinal_Processing/tp2_feature_FLAIR.nii.gz")
print()
file = open(masterfile, 'w')
file.write(f"{locate_training_directory}/Longitudinal_Processing/tp2_feature_FLAIR.nii.gz")
file.close()

masterfile=masterfile.replace(" ", "\ ")
bianca_output = "tp2_BIANCA_LPM.nii.gz"

print(f"cd {locate_parent_directory} && bianca --singlefile={masterfile} --loadclassifierdata={locate_parent_directory}/mytraining --querysubjectnum=1 --brainmaskfeaturenum=1 -o {locate_training_directory}/Longitudinal_Processing/tp2_BIANCA_LPM.nii.gz -v")
print()

p = os.popen(
    f"cd {locate_parent_directory} && bianca --singlefile={masterfile} --loadclassifierdata={locate_parent_directory}/mytraining --querysubjectnum=1 --brainmaskfeaturenum=1 -o {locate_training_directory}/Longitudinal_Processing/tp2_BIANCA_LPM.nii.gz -v")
if p:
    output = p.read()
    print(output)
    
# Storing them appropriately
output = run_command(f"cd {locate_training_directory} && mv {locate_training_directory}/Longitudinal_Processing/tp2_feature_FLAIR.nii.gz {testing_directory}/tp2_feature_FLAIR.nii.gz")

output = run_command(f"cd {locate_training_directory} && mv {locate_training_directory}/Longitudinal_Processing/tp2_feature_FLAIR_bianca_mask.nii.gz {testing_directory}/tp2_biancamask.nii.gz")

output = run_command(f"cd {locate_training_directory} && mv {locate_training_directory}/Longitudinal_Processing/tp2_feature_FLAIR_brain_mask.nii.gz {testing_directory}/tp2_brainmask.nii.gz")

output = run_command(f"cd {locate_training_directory} && mv {locate_training_directory}/Longitudinal_Processing/tp2_ventdistmap.nii.gz {testing_directory}/tp2_ventdistmap.nii.gz")

output = run_command(f"cd {locate_training_directory} && mv {locate_training_directory}/Longitudinal_Processing/tp2_BIANCA_LPM.nii.gz {testing_directory}/tp2_BIANCA_LPM.nii.gz")


/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02/LOCATE_Training-2023-04-02/Longitudinal_Processing/tp2_feature_FLAIR.nii.gz

cd /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02 && bianca --singlefile=/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02/LOCATE_Training-2023-04-02/Longitudinal_Processing/tp2_masterfile.txt --loadclassifierdata=/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02/mytraining --querysubjectnum=1 --brainmaskfeaturenum=1 -o /Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02/LOCATE_Training-2023-04-02/Longitudinal_Processing/tp2_BIANCA_LPM.nii.gz -v

Number of modalities = 1 , number of possible training subjects = 1
Files are: label file list = , data file li

Having run the above blocks of code, we should have the following directory structure:

- Insert Image of Training Folder with the Training Imgs and Longitudinal Tests Folders highlighted

## Step 5: Run the LOCATE Training Model

As we have all the files we need to run the LOCATE Model on, we will go ahead and do that!

In [53]:
print(f"/Applications/MATLAB_R2022b.app/bin/matlab -nodisplay -r \"cd('{locate_training_directory}'); LOCATE_testing('Longitudinal_Tests', 'Training_imgs_directory');exit\"")

output = run_command(f"/Applications/MATLAB_R2022b.app/bin/matlab -nodisplay -r \"cd('{locate_training_directory}'); LOCATE_testing('Longitudinal_Tests', 'Training_imgs_directory');exit\"")


/Applications/MATLAB_R2022b.app/bin/matlab -nodisplay -r "cd('/Users/zunairviqar/Documents/GitHub/MS-Longitudinal-Analysis/Fall_2023/Sample_Prepared_Data/Training-2023-04-02/LOCATE_Training-2023-04-02'); LOCATE_testing('Longitudinal_Tests', 'Training_imgs_directory');exit"


The output will be stored in the Longitudinal Tests directory. Particularly within the 'LOCATE_results_directory' and should be named something similar to:

- ms-010-tp1_BIANCA_LOCATE_binarylesionmap.nii.gz
- ms-010-tp2_BIANCA_LOCATE_binarylesionmap.nii.gz

## Step 6: Clean Intermediary files

Since we do not need the files in the intermediary folders, we will go ahead and remove them!

In [56]:
import shutil
shutil.rmtree(processing_directory)

# Analysis & Result 

### As part of our results, we have to obtain 3 different files:

- New Lesions that appeared in the latest time point

In [58]:
output = run_command(f"cd {testing_directory}/LOCATE_results_directory && fslmaths tp2_BIANCA_LOCATE_binarylesionmap.nii.gz -sub tp1_BIANCA_LOCATE_binarylesionmap.nii.gz {locate_training_directory}/Longitudinal_Tests/new_lesion_mask.nii.gz")


- All Lesions that ever existed

In [59]:
output = run_command(f"cd {testing_directory}/LOCATE_results_directory && fslmaths tp2_BIANCA_LOCATE_binarylesionmap.nii.gz -add tp1_BIANCA_LOCATE_binarylesionmap.nii.gz {locate_training_directory}/Longitudinal_Tests/all_lesion_mask.nii.gz")


- Lesions that were only in the previous time point

In [60]:
output = run_command(f"cd {testing_directory}/LOCATE_results_directory && fslmaths tp1_BIANCA_LOCATE_binarylesionmap.nii.gz -sub tp2_BIANCA_LOCATE_binarylesionmap.nii.gz {locate_training_directory}/Longitudinal_Tests/lost_lesion_mask.nii.gz")


### Then, we calculate the Statistics

In [79]:
# As per https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/BIANCA/Userguide#Performance_evaluation, 
# The command to run is:

# bianca_cluster_stats <bianca_output_map> <threshold> <min_cluster_size> [<mask>]

output = run_command(f"cd {testing_directory}/LOCATE_results_directory && bianca_cluster_stats tp1_BIANCA_LOCATE_binarylesionmap.nii.gz 0 1")
print(output)

num_lesions_1 = output.split("\n")[0].split(" ")[-1]
lesions_vol_1 = output.split("\n")[1].split(" ")[-1]


output = run_command(f"cd {testing_directory}/LOCATE_results_directory && bianca_cluster_stats tp2_BIANCA_LOCATE_binarylesionmap.nii.gz 0 1")
print(output)

num_lesions_2 = output.split("\n")[0].split(" ")[-1]
lesions_vol_2 = output.split("\n")[1].split(" ")[-1]

print("The Number of WMH has increased/decreased by :",int(num_lesions_2)-int(num_lesions_1))
print("The Total Volume has increased/decreased by :",float(lesions_vol_2)-float(lesions_vol_1))


 total WMH number for tp1_BIANCA_LOCATE_binarylesionmap.nii.gz with threshold 0 and minimum cluster size 1 is 188
 total volume for tp1_BIANCA_LOCATE_binarylesionmap.nii.gz with threshold 0 and minimum cluster size 1 is 3351.452637

 total WMH number for tp2_BIANCA_LOCATE_binarylesionmap.nii.gz with threshold 0 and minimum cluster size 1 is 205
 total volume for tp2_BIANCA_LOCATE_binarylesionmap.nii.gz with threshold 0 and minimum cluster size 1 is 8448.857422

The Number of WMH has increased/decreased by : 17
The Total Volume has increased/decreased by : 5097.404784999999
